<a href="https://colab.research.google.com/github/coldsober-irene/ASSIGNMENTS/blob/main/kivyapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

# Define the system matrices
A = np.array([[0.9638, 0.0162, 0, 0],
              [0.0162, 0.9627, 0, 0],
              [0.9638, 0.0162, 1.0000, 0],
              [0.0162, 0.9627, 0, 1.0000]])

B = np.array([[0.0031, 0],
              [0, 0.0031],
              [0.0031, 0],
              [0, 0.0031]])

C = np.array([[0, 0, 1, 0],
              [0, 0, 0, 1]])

# Define the system dimensions
n = A.shape[0]  # Number of states
m = B.shape[1]  # Number of inputs

# Define the prediction horizon
N = 10

# Define the control horizon
Nc = 3

# Define the initial state
x0 = np.zeros((n, 1))

# Define the reference input and states
xr = np.array([[15]* (N+1), [18]* (N+1)])  # Reference state (H1 = 15 cm, H2 = 18 cm)
ur = np.zeros((m, N))    # Reference input

# Define decision variables
X = cp.Variable((n, N+1))  # State trajectory variables
U = cp.Variable((m, N))    # Control trajectory variables

# Define the reference output
yr = np.array([[15]* (N+1), [18]* (N+1)])  # Reference output (H1 = 15 cm, H2 = 18 cm)

# Compute the predicted output
Y = C @ X
# Define weights
Q = np.eye(C.shape[0])  # Weight on output error
R = 0.1 * np.eye(m)  # Weight on control effort
# Define the cost function
cost = cp.quad_form(Y[:, :N] - yr[:, :N], Q) + cp.quad_form(U[:, :Nc] - ur[:, :Nc], R)


# Define the constraints
constr = [X[:, 0] == x0[:, 0]]
for i in range(N):
    constr += [X[:, i+1] == A@X[:, i] + B@U[:, i]]  # System dynamics
    constr += [cp.norm(U[:, i], 'inf') <= 35.1363]  # Control constraints for Motor1
    constr += [cp.norm(U[:, i], 'inf') <= 34.1162]  # Control constraints for Motor2
    constr += [cp.norm(X[:, i], 'inf') <= 18.23]  # State constraints for Tank1
    constr += [cp.norm(X[:, i], 'inf') <= 20.85]  # State constraints for Tank2
    if i < N - 1:
        constr += [cp.norm(U[:, i+1] - U[:, i], 'inf') <= 5]  # Change in control constraints

# Define the problem and solve
prob = cp.Problem(cp.Minimize(cost), constr)
prob.solve()

# Get the optimal state and control sequences
X_opt = X.value
U_opt = U.value

# Compute the plant output
Y_opt = np.zeros((C.shape[0], N+1))
for i in range(N):
    Y_opt[:, i] = C @ X_opt[:, i]

# Create a time vector
t = np.linspace(0, N, N+1)

# Plot the incremental control input
plt.figure()
plt.step(t[:-1], U_opt[0, :], where='post')
plt.title('Incremental Control Input U')
plt.xlabel('Sampling Time Steps')
plt.ylabel('Control Input')
plt.grid(True)

# Plot the plant output
plt.figure()
plt.step(t, Y_opt[0, :], where='post')
plt.title('Plant Output Y')
plt.xlabel('Sampling Time Steps')
plt.ylabel('Output')
plt.grid(True)

plt.show()


ValueError: ignored

In [ ]:
# Get the optimal state and control sequences
X_opt = X.value
U_opt = U.value

# Compute the plant output
Y_opt = np.zeros((C.shape[0], N+1))
for i in range(N):
    Y_opt[:, i] = C @ X_opt[:, i]

# Create a time vector
t = np.linspace(0, N, N+1)

# Plot the incremental control input
plt.figure()
plt.step(t[:-1], U_opt[0, :], where='post')
plt.title('Incremental Control Input U')
plt.xlabel('Sampling Time Steps')
plt.ylabel('Control Input')
plt.grid(True)

# Plot the plant output
plt.figure()
plt.step(t, Y_opt[0, :], where='post')
plt.title('Plant Output Y')
plt.xlabel('Sampling Time Steps')
plt.ylabel('Output')
plt.grid(True)

plt.show()


In [8]:
import cvxpy as cp
import numpy as np

# Define the system matrices
A = np.array([[1, 1], [0, 1]])
B = np.array([[0.5], [1]])
T = 10
# Define the state and input variables
x = cp.Variable((2, T+1))
u = cp.Variable((1, T))

# Define the initial state
x0 = np.array([1, 0])

# Define the state constraints
x_min = np.array([-10, -10])
x_max = np.array([10, 10])

# Define the input constraints
u_min = np.array([-1])
u_max = np.array([1])

# Define the cost function
cost = cp.sum_squares(x) + cp.sum_squares(u)

# Define the constraints
constraints = [x[:,0] == x0]
for t in range(T):
    constraints += [x[:,t+1] == A @ x[:,t] + B @ u[:,t],
                    x_min <= x[:,t+1], x[:,t+1] <= x_max,
                    u_min <= u[:,t], u[:,t] <= u_max]

# Define the problem and solve it
problem = cp.Problem(cp.Minimize(cost), constraints)
problem.solve()

# The optimal input sequence is then given by u.value


2.367101283098448

In [9]:
import numpy as np
from scipy.linalg import expm

# Define the system matrices
A = np.array([[0.9838, 0.0162], [0.0162, 0.9827]])  # replace "..." with your values
B = np.array([[0.0031, 0], [0, 0.0031]])

# Define the sampling time
Ts = 0.1  # replace with your value

# Define the prediction horizon
N = 10  # replace with your value

# Define the initial state
x_k = np.array([1, 0])  # replace with your values

# Calculate the matrix F
F = np.zeros((2*N, 2))
for i in range(N):
    F[2*i:2*(i+1), :] = np.dot(expm(A*Ts*i), B)

# Calculate the matrix theta
theta = np.zeros((2*N, 2*N))
for i in range(N):
    for j in range(i+1):
        theta[2*i:2*(i+1), 2*j:2*(j+1)] = expm(A*Ts*(i-j))

# Now you can calculate the predicted output
Y = np.dot(F, x_k) + np.dot(theta, Delta_U)


NameError: ignored

In [12]:
!pip install toga

  Using cached toga-0.4.1-py3-none-any.whl (3.3 kB)
  Using cached toga_gtk-0.4.1-py3-none-any.whl (54 kB)
  Using cached gbulb-0.6.4-py3-none-any.whl (17 kB)
  Using cached pycairo-1.25.1.tar.gz (347 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyGObject-3.46.0.tar.gz (723 kB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [10]:
import toga
from toga.style import Pack
from toga.style.pack import COLUMN, ROW

class SignUpApp(toga.App):
    def startup(self):
        # Create a main window with a title
        self.main_window = toga.MainWindow(title=self.name)

        # Create a box to hold the widgets
        box = toga.Box(style=Pack(direction=COLUMN))

        # Create a label to display the title
        title_label = toga.Label(
            'Sign up',
            style=Pack(padding=(0, 5))
        )
        box.add(title_label)

        # Create a box to hold the input fields
        input_box = toga.Box(style=Pack(direction=ROW, padding=5))

        # Create a label and a text input for the name
        name_label = toga.Label(
            'Name:',
            style=Pack(width=100, padding_right=5)
        )
        name_input = toga.TextInput(style=Pack(flex=1))

        # Add the label and the input to the input box
        input_box.add(name_label)
        input_box.add(name_input)

        # Add the input box to the main box
        box.add(input_box)

        # Repeat the same steps for the email and the password
        # ...

        # Create a button to submit the form
        sign_up_button = toga.Button(
            'Sign up',
            on_press=self.sign_up,
            style=Pack(padding=5)
        )
        box.add(sign_up_button)

        # Add the main box to the main window
        self.main_window.content = box

        # Show the main window
        self.main_window.show()

    def sign_up(self, widget):
        # Get the values from the input fields
        name = name_input.value
        email = email_input.value
        password = password_input.value

        # Validate the input and show a message
        if name and email and password:
            message = f'Hello, {name}! You have signed up successfully.'
        else:
            message = 'Please fill in all the fields.'

        self.main_window.info_dialog('Sign up', message)


def main():
    return SignUpApp('Sign Up', 'org.beeware.sign-up')

ModuleNotFoundError: ignored

In [ ]:
!sudo apt update

In [14]:
!sudo apt install -y git zip unzip openjdk-8-jdk python3-pip autoconf libtool pkg-config zlib1g-dev libncurses5-dev libncursesw5-dev libtinfo5 cmake libffi-dev libssl-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
autoconf is already the newest version (2.71-2).
autoconf set to manually installed.
pkg-config is already the newest version (0.29.2-1ubuntu3).
zip is already the newest version (3.0-12build2).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
git is already the newest version (1:2.34.1-1ubuntu1.10).
libncurses5-dev is already the newest version (6.3-2ubuntu0.1).
libssl-dev is already the newest version (3.0.2-0ubuntu1.12).
unzip is already the newest version (6.0-26ubuntu3.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
zlib1g-dev set to manually installed.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common libsm-dev
  libxkbfile1 libxt-dev libxtst6 libxxf86d

In [15]:
!pip3 install --upgrade Cython==0.29.19 virtualenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 971.9/971.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.7
    Uninstalling Cython-3.0.7:
      Successfully uninstalled Cython-3.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires cython>=0.29.21, but you have cython 0.29.19 which is incompatible.


In [16]:
!export PATH=$PATH:~/.local/bin/

In [17]:
!pip install buildozer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 1.4 MB/s eta 0:00:00


In [18]:
!sudo apt-get install -y \
    python3-pip \
    build-essential \
    git \
    python3 \
    python3-dev \
    ffmpeg \
    libsdl2-dev \
    libsdl2-image-dev \
    libsdl2-mixer-dev \
    libsdl2-ttf-dev \
    libportmidi-dev \
    libswscale-dev \
    libavformat-dev \
    libavcodec-dev \
    zlib1g-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
git is already the newest version (1:2.34.1-1ubuntu1.10).
python3 is already the newest version (3.10.6-1~22.04).
python3 set to manually installed.
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavcodec-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavcodec-dev set to manually installed.
libavformat-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavformat-dev set to manually installed.
libswscale-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libswscale-dev set to manually installed.
python3-pip is already the newest version (22.0.2+dfsg-1ubuntu0.4).
The following additional packages will be ins

In [19]:
!sudo apt-get install -y \
    python3-pip \
    build-essential \
    git \
    python3 \
    python3-dev \
    ffmpeg \
    libsdl2-dev \
    libsdl2-image-dev \
    libsdl2-mixer-dev \
    libsdl2-ttf-dev \
    libportmidi-dev \
    libswscale-dev \
    libavformat-dev \
    libavcodec-dev \
    zlib1g-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libportmidi-dev is already the newest version (1:217-6).
libsdl2-image-dev is already the newest version (2.0.5+dfsg1-3build1).
libsdl2-mixer-dev is already the newest version (2.0.4+dfsg1-4build1).
libsdl2-ttf-dev is already the newest version (2.0.18+dfsg-2).
git is already the newest version (1:2.34.1-1ubuntu1.10).
python3 is already the newest version (3.10.6-1~22.04).
python3-dev is already the newest version (3.10.6-1~22.04).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavcodec-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavformat-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libsdl2-dev is already the newest version (2.0.20+dfsg-2ubuntu1.22.04.1).
libswscale-dev is already the newest version (7:4.4

In [21]:
!sudo apt-get install build-essential libsqlite3-dev sqlite3 bzip2 libbz2-dev zlib1g-dev libssl-dev openssl libgdbm-dev libgdbm-compat-dev liblzma-dev libreadline-dev libncursesw5-dev libffi-dev uuid-dev libffi6

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libffi6


In [22]:
!sudo apt-get install libffi-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libffi-dev is already the newest version (3.4.2-4).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [23]:
!buildozer init

Buildozer is running as root!
This is not recommended, and may lead to problems later.
Are you sure you want to continue [y/n]? y
File buildozer.spec created, ready to customize!


In [25]:
!git clone https://github.com/kivy/python-for-android.git

Cloning into 'python-for-android'...
remote: Enumerating objects: 26815, done.
remote: Counting objects: 100% (595/595), done.
remote: Compressing objects: 100% (366/366), done.
remote: Total 26815 (delta 273), reused 445 (delta 185), pack-reused 26220
Receiving objects: 100% (26815/26815), 18.46 MiB | 8.15 MiB/s, done.
Resolving deltas: 100% (14584/14584), done.


In [ ]:
!buildozer -v android debug